In [13]:
from parquet_data_reader import ParquetDataReader
import polars as pl
import pandas as pd
import numpy as np
from utils.process_data import process_item_data, explode_user_interactions
from parquet_data_reader import ParquetDataReader
from models.feature_based import CosineSimilarityRecommender
pl.Config.set_tbl_cols(-1)

polars.config.Config

In [14]:
article_df = pl.read_parquet("../generated_data/train/items.parquet")


In [15]:

article_df = process_item_data(article_df)

article_df.write_parquet("../generated_data/train/items_processed.parquet")

In [16]:
article_df = pl.read_parquet("../generated_data/train/items_processed.parquet")


In [17]:
data_reader = ParquetDataReader()

train_behaviors_df = data_reader.read_data('../data/train/behaviors.parquet')
train_history_df = data_reader.read_data('../data/train/history.parquet')

In [18]:
train_history_exploaded_df = explode_user_interactions(train_history_df)


In [19]:
train_history_exploaded_df.describe()

statistic,user_id,impression_time,scroll_percentage,article_id,read_time
str,f64,str,f64,f64,f64
"""count""",2.426247e6,"""2426247""",2.171171e6,2.426247e6,2.426247e6
"""null_count""",0.0,"""0""",255076.0,0.0,0.0
"""mean""",1.3074e6,"""2023-05-08 01:11:34.336897""",68.223557,9.7403e6,61.370533
"""std""",738740.392112,null,32.312428,178068.888024,165.437134
"""min""",10068.0,"""2023-04-27 07:00:00""",0.0,3.001353e6,0.0
"""25%""",671325.0,"""2023-05-02 18:06:30""",35.0,9.744897e6,5.0
"""50%""",1.314594e6,"""2023-05-08 06:55:34""",77.0,9.752375e6,16.0
"""75%""",1.950222e6,"""2023-05-13 09:50:20""",100.0,9.761768e6,51.0
"""max""",2.590637e6,"""2023-05-18 06:59:59""",100.0,9.771042e6,1800.0


In [20]:
#user_item_features_df = pl.read_parquet("../generated_data/train/history_exploaded.parquet")

#user_item_features_df = user_item_features_df.join(article_df, on="article_id", how="inner")

#user_item_features_df.write_parquet("../generated_data/train/history_exploaded_with_item_features.parquet")

#user_item_features_df.head()

In [21]:
user_item_features_df = pl.read_parquet("../generated_data/train/history_exploaded_with_item_features.parquet")

In [22]:
cosineSimilarityRecommender = CosineSimilarityRecommender(user_item_features_df)

cosineSimilarityRecommender.fit()

cosineSimilarityRecommender.recommend(13538, 10)

/home/pedropca/Documents/Datatek/Recommender systems/TDT4215/recommender_system/models/feature_based.py:88: UserWarning: encountered expression deeper than 512 elements; this may overflow the stack, consider refactoring
  pairs = pairs.with_columns(


article_id,score
i32,f32
9751982,0.675091
9765410,0.646043
9712680,0.631963
9739634,0.616901
9600582,0.602545
9740236,0.6013
9737245,0.600413
9725707,0.598653
9742559,0.597831


In [23]:
prediction = cosineSimilarityRecommender.predict(13538, 9751982)

print(prediction)

0.6750908123731939


In [24]:
prediction = cosineSimilarityRecommender.predict(13538, 9765630)

print(prediction)

0.5930624391050546
